In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import copy
import sys
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
from matplotlib import animation
from IPython.display import HTML
import statistics

In [ ]:
worldSize=100 # Side length of square grid world 
nFires=200 # Number of fires (approximately)
nDrones=3 # Number of Drones
world=[] # Actual world 
drones=[] # List of Drones with their information
worldMap=[] # View of the world from the drones' perspective
nGrids=[] # List of (bottom left, top right) search bounds for each drone
mode='search' # mode of the system - search and map 
borders=[]
mapTargets=[]
adjList = []
perimeter = [] 
firesDiscovered = []
allFires = []
currFrame = 1
ros_ref_points = []
ros_estimates = []
actual_ros_ref_points = []
actual_ros_estimates = []
perimeter_coverage = []
soft_perimeter_coverage = []
efficiencies = []
last_ros_update = 0
last_actual_ros_update = 0


In [ ]:
km_per_square = 0.1 #in km
frame_time_millis = 100;
millis_per_hr = 60*60*1000
speedup_factor = 100
drone_speed = km_per_square / ((frame_time_millis*speedup_factor)/(millis_per_hr)) #drone speed
frames_per_sim_hr = (millis_per_hr)/(frame_time_millis * speedup_factor)
windSpeed = 20 #assuming it goes north
fuel_combustability=1
ellipse_ratio = 1 + 0.00120*windSpeed**2.154
ros_head = fuel_combustability*((windSpeed/10)/km_per_square)   #in squares/hr
ros_flanks = ros_head/(2*ellipse_ratio)   #in squares/hr 
#ros_head = expected downwind spread at given column, per hour in simulation = (# fire spread updates per hour in simulation)*prob(square spreads in downwind direction) 
#                                 prob(square spreads in downwind direction) = ros_head/(# fire spread updates per hour within simulation)
#                                                                            = ros_head/frames_per_sim_hr)
prob_spread_head = ros_head/(frames_per_sim_hr)  
prob_spread_flanks = ros_flanks/(frames_per_sim_hr)
spreadMode = 'ellipse'

In [ ]:
def batch_run():

  global nDrones
  global nFires
  global windSpeed
  global perimeter_coverage
  global soft_perimeter_coverage
  global efficiencies
  global ellipse_ratio
  global ros_head
  global ros_flanks
  global prob_spread_head
  global prob_spread_flanks
  global speedup_factor

  numSims = 10
  #varying parameters
  #windSpeed_params = list(np.linspace(0,50,num = 10)
  nDrones_params = np.linspace(2,10,5)
  #nFires_params = np.linspace(100,500,10)
  #fuel_combustability=1

  drone_avg_efficiencies_allsims = np.zeros((5,1))
  drone_stddev_efficiencies_allsims = np.zeros((5,1))
  max_perimeter_coverage_allsims = np.zeros((5,1))
  avg_perimeter_coverage_allsims = np.zeros((5,1))
  avg_soft_perimeter_coverage_allsims = np.zeros((5,1))
  all_stats = []
  windspeeds = [20,30,40]
  for wind in windspeeds:
    for i in range(len(nDrones_params)):
      for j in range(numSims):
        print(str(wind) + " " + str(int(nDrones_params[i])) + " " + str(j))
        initSimParams()
        nDrones = int(nDrones_params[i])
        windSpeed = wind
        ellipse_ratio = 1 + 0.00120*windSpeed**2.154
        ros_head = fuel_combustability*((windSpeed/10)/km_per_square)   #in squares/hr
        ros_flanks = ros_head/(2*ellipse_ratio)   #in squares/hr 
        prob_spread_head = ros_head/(frames_per_sim_hr)  
        prob_spread_flanks = ros_flanks/(frames_per_sim_hr)
        numFrames = int(1/(frame_time_millis*speedup_factor/millis_per_hr)) #simulation will last 1 hour
        print(numFrames)
        sim_without_plots(numFrames)
        drone_avg_efficiencies_allsims[i] += (1/numSims)*efficiency_stats()[0]
        drone_stddev_efficiencies_allsims[i] += (1/numSims)*efficiency_stats()[1] 
        max_perimeter_coverage_allsims[i] += (1/numSims)*(max(perimeter_coverage)) 
        avg_perimeter_coverage_allsims[i] += (1/numSims)*sum([i for i in perimeter_coverage if i != 0])/len([i for i in perimeter_coverage if i != 0])
        avg_soft_perimeter_coverage_allsims[i] += (1/numSims)*sum([i for i in soft_perimeter_coverage if i != 0])/len([i for i in soft_perimeter_coverage if i !=0])
        print("highest fire: " + str((max(firesDiscovered, key=lambda x: x[1])))) #checking that fire didn't grow to border of map
        print("rightmost fire: " + str((max(firesDiscovered, key=lambda x: x[0]))))
        print("leftmost fire: " + str((min(firesDiscovered, key=lambda x: x[0]))))

    all_stats.append([drone_avg_efficiencies_allsims,drone_stddev_efficiencies_allsims,max_perimeter_coverage_allsims,avg_perimeter_coverage_allsims,avg_soft_perimeter_coverage_allsims])
    drone_avg_efficiencies_allsims = np.zeros((5,1))
    drone_stddev_efficiencies_allsims = np.zeros((5,1))
    max_perimeter_coverage_allsims = np.zeros((5,1))
    avg_perimeter_coverage_allsims = np.zeros((5,1))
    avg_soft_perimeter_coverage_allsims = np.zeros((5,1))


  fig1, ax1 = plt.subplots()
  ax1.set_title("Max. Perimeter Coverage vs Number of Drones")
  ax1.set_xlabel("Number of drones")  
  ax1.set_ylabel("Max. % of perimeter covered")
  colors = ['b','r','g']
  print("length: " + str(len(all_stats)))
  for i in range(len(all_stats)):
    ax1.scatter(list(nDrones_params), all_stats[i][2], label="Wind = " + str(windspeeds[i]) + " km/h", color = colors[i])
    ax1.plot(list(nDrones_params),all_stats[i][2], color = colors[i])
  ax1.legend(loc='upper left');
  plt.show()

  fig2, ax2 = plt.subplots()
  ax2.set_title("Avg. Perimeter Coverage vs Number of Drones")
  ax2.set_xlabel("Number of drones")  
  ax2.set_ylabel("Avg. % of perimeter covered")
  colors = ['b','r','g']
  print("length: " + str(len(all_stats)))
  for i in range(len(all_stats)):
    ax2.scatter(list(nDrones_params), all_stats[i][3], label="Wind = " + str(windspeeds[i]) + " km/h", color = colors[i])
    ax2.plot(list(nDrones_params),all_stats[i][3], color = colors[i])
  ax2.legend(loc='upper left');
  plt.show()

  fig3, ax3 = plt.subplots()
  ax3.set_title("Avg. Standard Deviation of Drone Efficiency vs Number of Drones")
  ax3.set_xlabel("Number of drones")  
  ax3.set_ylabel("Avg. standard deviation of drone efficiency")
  colors = ['b','r','g']
  print("length: " + str(len(all_stats)))
  for i in range(len(all_stats)):
    ax3.scatter(list(nDrones_params), all_stats[i][1], label="Wind = " + str(windspeeds[i]) + " km/h", color = colors[i])
    ax3.plot(list(nDrones_params),all_stats[i][1], color = colors[i])
  ax3.legend(loc='upper right');

  fig4, ax4 = plt.subplots()
  ax4.set_title("Avg. Drone Efficiency vs Number of Drones")
  ax4.set_xlabel("Number of drones")  
  ax4.set_ylabel("Avg. standard deviation of drone efficiency")
  colors = ['b','r','g']
  print("length: " + str(len(all_stats)))
  for i in range(len(all_stats)):
    ax4.scatter(list(nDrones_params), all_stats[i][0], label="Wind = " + str(windspeeds[i]) + " km/h", color = colors[i])
    ax4.plot(list(nDrones_params),all_stats[i][0], color = colors[i])
  ax4.legend(loc='upper right');

  fig5, ax5 = plt.subplots()
  ax5.set_title("Avg. Soft Perimeter Coverage vs Number of Drones")
  ax5.set_xlabel("Number of drones")  
  ax5.set_ylabel("Avg. % of soft perimeter covered")
  colors = ['b','r','g']
  print("length: " + str(len(all_stats)))
  for i in range(len(all_stats)):
    ax5.scatter(list(nDrones_params), all_stats[i][4], label="Wind = " + str(windspeeds[i]) + " km/h", color = colors[i])
    ax5.plot(list(nDrones_params),all_stats[i][4], color = colors[i])
  ax5.legend(loc='upper left');


  for i in all_stats:
    print(i)
  # print(drone_avg_efficiencies_allsims)
  # print(drone_stddev_efficiencies_allsims)
  # print(max_perimeter_coverage_allsims)
  # print(avg_perimeter_coverage_allsims)

        
  

        
  



In [ ]:
def efficiency_stats():
  global efficiencies
  averages = []
  std_devs = []
  for i in efficiencies:
    avg = statistics.mean([j[1] for j in i])
    if avg != 0: # == 0 when drones haven't detected any fire yet
      averages.append(avg)
      std_devs.append(statistics.stdev([j[1] for j in i]))
  
  overall_avg = statistics.mean(averages)
  avg_std_dev = statistics.mean(std_devs)
  return [overall_avg,avg_std_dev]



In [ ]:
def sim_without_plots(numFrames):
  createWorld()
  spawnDrones()
  createWorldMap()
  worldSplit()
  setInitTargets()
  for i in range(numFrames):
    #print(str(i) + "/" + str(numFrames))
    updateWorld()
    percent_perimeter_detected()
    percent_soft_perimeter_detected()
    calc_drone_efficiencies()
    estimate_ros()
    actual_ros()
    

In [ ]:
def initSimParams():
  global worldSize
  global nFires
  global nDrones
  global world
  global drones
  global worldMap 
  global nGrids
  global mode
  global borders
  global mapTargets
  global adjList
  global perimeter
  global firesDiscovered
  global currFrame
  global ros_ref_points
  global ros_estimates
  global perimeter_coverage
  global soft_perimeter_coverage
  global efficiencies
  global last_ros_update
  global km_per_square
  global frame_time_millis 
  global millis_per_hr 
  global speedup_factor
  global drone_speed
  global frames_per_sim_hr 
  global windSpeed 
  global fuel_combustability
  global ellipse_ratio 
  global ros_head
  global ros_flanks  
  global prob_spread_head
  global prob_spread_flanks
  global actual_ros_ref_points
  global actual_ros_estimates
  global last_actual_ros_update
  global allFires
  global spreadMode
  global startingFire

  worldSize=100 # Side length of square grid world 
  nFires=200 # Number of fires (approximately)
  nDrones=3 # Number of Drones
  world=[] # Actual world 
  drones=[] # List of Drones with their information
  worldMap=[] # Viewof the world from the drones' perspective
  nGrids=[] # List of (bottom left, top right) of search bounds for each drone
  mode='search' # mode of the system - search and map 
  borders=[]
  mapTargets=[]
  adjList = []
  perimeter = [] 
  firesDiscovered = []
  allFires = []
  currFrame = 1
  ros_ref_points = []
  ros_estimates = []
  actual_ros_ref_points = []
  actual_ros_estimates = []
  perimeter_coverage = []
  soft_perimeter_coverage = []
  efficiencies = []
  last_ros_update = 0
  last_actual_ros_update = 0
  startingFire = []

  km_per_square = 0.25 #in km
  frame_time_millis = 100
  millis_per_hr = 60*60*1000
  speedup_factor = 200
  drone_speed = km_per_square / ((frame_time_millis*speedup_factor)/(millis_per_hr)) #drone speed
  frames_per_sim_hr = (millis_per_hr)/(frame_time_millis * speedup_factor)
  windSpeed = 50 #assuming it goes north
  fuel_combustability=1
  ellipse_ratio = 1 + 0.00120*windSpeed**2.154
  ros_head = fuel_combustability*((windSpeed/10)/km_per_square)   #in squares/hr
  ros_flanks = ros_head/(2*ellipse_ratio)   #in squares/hr 
  prob_spread_head = ros_head/(frames_per_sim_hr)  
  prob_spread_flanks = ros_flanks/(frames_per_sim_hr)
  spreadMode = 'ellipse'

In [ ]:
def distance(x,y):
  return abs(x[0]-y[0])+abs(x[1]-y[1])

In [ ]:
def plot(grid):
  img=copy.deepcopy(grid)
  img=np.array(img).T.tolist()
  for i in range(len(img)):
    for j in range(len(img)):
      if img[i][j]==1 or img[i][j]=='1': # Fire
        img[i][j]=[255,0,0] # Red
      elif type(img[i][j])==str and img[i][j][0]=='D': # Drone ex: D0, D1
        img[i][j]=[0,0,255] # Blue
      elif type(img[i][j])==str and img[i][j][0]=='T': # Target ex: T0, T1
        img[i][j]=[0,255,255] # Cyan
      elif type(img[i][j])==str and img[i][j][0]=='V': # V -> Visited ex: V0, V1
        img[i][j]=[150,150,150] # Dark Gray
      elif type(img[i][j])==str and img[i][j][0]=='S': # S -> Seen ex: S0, S1
        img[i][j]=[210,210,210] # Light Gray
      elif type(img[i][j])==str and img[i][j][0]=='B': # B -> Border ex: B0, B1
        img[i][j]=[0,255,0] # Green
      else:
        img[i][j]=[255,255,255] # White
  plt.imshow(img,origin="lower")
  ax=plt.gca()
  #ax.invert_yaxis()
  plt.show()

In [ ]:
# def createWorld():  
#   global world
#   global worldMap
#   global adjList
#   world=[]
#   for i in range(worldSize):
#     world.append([])
#     for j in range(worldSize):
#       world[i].append(0)

#   adjList=[]
#   #startX=random.randrange(0,worldSize)
#   #startY=random.randrange(0,worldSize)
#   startX=random.randrange(worldSize//8,7*worldSize//8)
#   startY=random.randrange(worldSize//8,7*worldSize//8)
#   #startX=worldSize//2
#   #startY=worldSize//2
#   world[startX][startY]=1
#   if startX>0:
#     adjList.append([startX-1,startY])
#   if startX<worldSize-1:
#     adjList.append([startX+1,startY])
#   if startY>0:
#     adjList.append([startX,startY-1])
#   if startY<worldSize-1:
#     adjList.append([startX,startY+1])
#   for i in range(nFires-1):
#     newFire=adjList[random.randrange(0,len(adjList))]
#     world[newFire[0]][newFire[1]]=1
#     adjList.remove(newFire)
#     if newFire[0]>1 and world[newFire[0]-1][newFire[1]]==0 and [newFire[0]-1,newFire[1]] not in adjList:
#       adjList.append([newFire[0]-1,newFire[1]])
#     if newFire[0]<worldSize-2 and world[newFire[0]+1][newFire[1]]==0 and [newFire[0]+1,newFire[1]] not in adjList:
#       adjList.append([newFire[0]+1,newFire[1]])
#     if newFire[1]>1 and world[newFire[0]][newFire[1]-1]==0 and [newFire[0],newFire[1]-1] not in adjList:
#       adjList.append([newFire[0],newFire[1]-1])
#     if newFire[1]<worldSize-2 and world[newFire[0]][newFire[1]+1]==0 and [newFire[0],newFire[1]+1] not in adjList:
#       adjList.append([newFire[0],newFire[1]+1])

#   npWorld=np.array(world)
#   for i in range(worldSize):
#     for j in range(worldSize):
#       if world[i][j]==0 and 1 in npWorld[0:i,j] and 1 in npWorld[i:worldSize,j] and 1 in npWorld[i,0:j] and 1 in npWorld[i,j:worldSize]:
#         world[i][j]=1
#   npWorld=np.array(world)
#   #plot(world)

In [ ]:
def createWorld():  
  global world
  global worldMap
  global adjList
  world=[]
  for i in range(worldSize):
    world.append([])
    for j in range(worldSize):
      world[i].append(0)

  #startX=random.randrange(0,worldSize)
  #startY=random.randrange(0,worldSize)
  startX=random.randrange(3*worldSize//8,6*worldSize//8)
  startY=random.randrange(2*worldSize//8,3*worldSize//8)
  #startX=worldSize//2
  #startY=worldSize//2
  world[startX][startY]=1
  startingFire = [startX,startY]
  if startX>0:
    adjList.append([startX-1,startY,prob_spread_flanks])
  if startX<worldSize-1:
    adjList.append([startX+1,startY,prob_spread_flanks])
  if startY>0:
    adjList.append([startX,startY-1,0])
  if startY<worldSize-1:
    adjList.append([startX,startY+1,prob_spread_head])
  updatePerimeter([startX,startY])
  allFires.append([startX,startY])


  if spreadMode == 'random':
    for i in range(nFires-1):
      newFire=adjList[random.randrange(0,len(adjList))]
      world[newFire[0]][newFire[1]]=1
      adjList.remove(newFire)
      allFires.append([newFire[0],newFire[1]])
      if newFire[0]>1 and world[newFire[0]-1][newFire[1]]!=1 and ([newFire[0]-1,newFire[1],prob_spread_head] not in adjList and [newFire[0]-1,newFire[1],prob_spread_flanks] not in adjList):
        adjList.append([newFire[0]-1,newFire[1],prob_spread_flanks])
      if newFire[0]<worldSize-2 and world[newFire[0]+1][newFire[1]]!=1 and ([newFire[0]+1,newFire[1],prob_spread_head] not in adjList and [newFire[0]+1,newFire[1],prob_spread_flanks] not in adjList):
        adjList.append([newFire[0]+1,newFire[1],prob_spread_flanks])
      if newFire[1]>1 and world[newFire[0]][newFire[1]-1]!=1 and ([newFire[0],newFire[1]-1, prob_spread_head] not in adjList and [newFire[0],newFire[1]-1,prob_spread_flanks] not in adjList):
        adjList.append([newFire[0],newFire[1]-1,0])
      if newFire[1]<worldSize-2 and world[newFire[0]][newFire[1]+1]!=1 and ([newFire[0],newFire[1]+1,prob_spread_head] not in adjList and [newFire[0],newFire[1]+1,prob_spread_flanks]):
        adjList.append([newFire[0],newFire[1]+1,prob_spread_head])
      updatePerimeter([newFire[0], newFire[1]])
      updatePerimeter([newFire[0]-1, newFire[1]])
      updatePerimeter([newFire[0]+1, newFire[1]])
      updatePerimeter([newFire[0], newFire[1]-1])
      updatePerimeter([newFire[0], newFire[1]+1])

  elif spreadMode == 'ellipse':
    fires_set = 1
    while fires_set != nFires - 1:
      tempAdjList = []
      for newFire in adjList:
        if fires_set != nFires - 1:
          #newFire=adjList[random.randrange(0,len(adjList))]
          randNum = random.random()
          if randNum <= newFire[2]:
            world[newFire[0]][newFire[1]]=1
            adjList.remove(newFire)
            allFires.append([newFire[0],newFire[1]])
            fires_set+=1
            if newFire[0]>1 and world[newFire[0]-1][newFire[1]]!=1 and ([newFire[0]-1,newFire[1],prob_spread_head] not in adjList and [newFire[0]-1,newFire[1],prob_spread_flanks] not in adjList):
              tempAdjList.append([newFire[0]-1,newFire[1],prob_spread_flanks])
            if newFire[0]<worldSize-2 and world[newFire[0]+1][newFire[1]]!=1 and ([newFire[0]+1,newFire[1],prob_spread_head] not in adjList and [newFire[0]+1,newFire[1],prob_spread_flanks] not in adjList):
              tempAdjList.append([newFire[0]+1,newFire[1],prob_spread_flanks])
            if newFire[1]>1 and world[newFire[0]][newFire[1]-1]!=1 and ([newFire[0],newFire[1]-1, prob_spread_head] not in adjList and [newFire[0],newFire[1]-1,prob_spread_flanks] not in adjList):
              tempAdjList.append([newFire[0],newFire[1]-1,0])
            if newFire[1]<worldSize-2 and world[newFire[0]][newFire[1]+1]!=1 and ([newFire[0],newFire[1]+1,prob_spread_head] not in adjList and [newFire[0],newFire[1]+1,prob_spread_flanks]):
              tempAdjList.append([newFire[0],newFire[1]+1,prob_spread_head])
            updatePerimeter([newFire[0], newFire[1]])
            updatePerimeter([newFire[0]-1, newFire[1]])
            updatePerimeter([newFire[0]+1, newFire[1]])
            updatePerimeter([newFire[0], newFire[1]-1])
            updatePerimeter([newFire[0], newFire[1]+1])
            
      adjList.extend(tempAdjList)

  npWorld=np.array(world)
  for i in range(worldSize):
    for j in range(worldSize):
      if world[i][j]==0 and 1 in npWorld[0:i,j] and 1 in npWorld[i:worldSize,j] and 1 in npWorld[i,0:j] and 1 in npWorld[i,j:worldSize]:
        world[i][j]=1
  npWorld=np.array(world)
  #plot(world)

In [ ]:
def spawnDrones():
  global mode
  global world
  global drones
  global borders
  global mapTargets
  mode="search"
  borders=[]
  mapTargets=[]
  drones=[]
  for i in range(nDrones):
    drones.append({})
    drones[i]['id']=i
    drones[i]['state']='goToTarget'
    drones[i]['currentlyAbove']='V'+str(i)
    drones[i]['border']=[]
    drones[i]['followCount']=0
    drones[i]['numDiscovered']=0

    ### This section spawns drones at random locations on the border of the map

    side=random.randrange(0,4)
    pos=random.randrange(0,worldSize)
    if side==0:
      world[0][pos]='D'+str(i)
      drones[i]['position']=[0,pos]
    elif side==1:
      world[pos][0]='D'+str(i)
      drones[i]['position']=[pos,0]
    elif side==2:
      world[worldSize-1][pos]='D'+str(i)
      drones[i]['position']=[worldSize-1,pos]
    else:
      world[pos][worldSize-1]='D'+str(i)
      drones[i]['position']=[pos,worldSize-1]
    
    ### This spawns all drones at origin [0,0]

    # drones[i]['position']=[0,0]

  npworld=np.array(world)
  #plot(world)

In [ ]:
def createWorldMap():
  global world
  global worldMap
  worldMap=copy.deepcopy(world)
  for i in range(worldSize):
    for j in range(worldSize):
      if worldMap[i][j]==1:
        worldMap[i][j]=0
  #plot(worldMap)

In [ ]:
# Not using this method
# Will be here if required in future 
def nearestDrone():
  searchSpace=[]
  droneSpaceCount=[0,0,0,0]
  for i in range(worldSize):
    searchSpace.append([])
    for j in range(worldSize):
      cur=[i,j]
      minDist=float('inf')
      for k in range(len(drones)):
        dist=distance(cur,drones[k])
        if dist<minDist:
          minDist=dist
          closestDrone=k
      if len(searchSpace[i])<worldSize:
        searchSpace[i].append(closestDrone)
      else:
        searchSpace[i][j]=closestDrone
      droneSpaceCount[closestDrone]=droneSpaceCount[closestDrone]+1
  searchSpace=np.array(searchSpace).T.tolist()
  plt.imshow(searchSpace, origin="lower")
  print(droneSpaceCount)

In [ ]:
def findSplit(bl,tr,n):
  div=int(n**0.5)
  if n==1:
    nGrids.append([bl,tr])
  elif tr[0]-bl[0] > tr[1]-bl[1]:
    splitPoint=int(bl[0]+(tr[0]-bl[0])*(div/n))
    findSplit(bl,[splitPoint,tr[1]],div)
    findSplit([splitPoint+1,bl[1]],tr,n-div)
  else:
    splitPoint=int(bl[1]+(tr[1]-bl[1])*(div/n))
    findSplit(bl,[tr[0],splitPoint],div)
    findSplit([bl[0],splitPoint+1],tr,n-div)

In [ ]:
def worldSplit():
  global nGrids
  g=[]
  for i in range(worldSize):
    g.append([])
    for j in range(worldSize):
      g[i].append(-1)
  nGrids=[]
  findSplit([0,0],[worldSize-1,worldSize-1],nDrones) #***made search bounds smaller
  label=0
  for i in nGrids:
    for j in range(i[0][0],i[1][0]+1):
      for k in range(i[0][1],i[1][1]+1):
        g[j][k]=label
    label=label+1
  plt.imshow(g,origin="lower")

In [ ]:
def setInitTargets():
  gridTargets=[]
  for j in nGrids:
    gridTargets.append([j[0][0]+1,j[0][1]+1])
  from scipy.spatial.distance import cdist
  from scipy.optimize import linear_sum_assignment
  targetDistances=[]
  for i in drones:
    targetDistances.append([])
    for j in gridTargets: 
      targetDistances[len(targetDistances)-1].append(distance(i['position'],j))
  _,assignment=linear_sum_assignment(targetDistances)
  for i in range(len(assignment)):
    drones[i]['target']=gridTargets[assignment[i]]
    drones[i]['searchBounds']=nGrids[assignment[i]]
  for i in drones:
    t=i['target']
    worldMap[t[0]][t[1]]='T'+str(i['id'])
  #plot(worldMap)

In [ ]:
def fireFound(drone):
  global mode
  global world 
  global worldMap
  global firesDiscovered
  loc=drone['position']
  id=str(drone['id'])
  visibility=[[True,True,True],[True,True,True],[True,True,True]]
  flag=False
  if loc[0]==0:
    for i in range(3):
      visibility[0][i]=False
  if loc[0]==worldSize-1:
    for i in range(3):
      visibility[2][i]=False
  if loc[1]==0:
    for i in range(3):
      visibility[i][0]=False
  if loc[1]==worldSize-1:
    for i in range(3):
      visibility[i][2]=False
  for i in range(3):
    for j in range(3):
      if visibility[i][j]:
        block=worldMap[loc[0]+i-1][loc[1]+j-1]
        if world[loc[0]+i-1][loc[1]+j-1]==1 and block!=1:
          drone['numDiscovered']+=1
          firesDiscovered.append([loc[0]+i-1, loc[1]+j-1])
        if world[loc[0]+i-1][loc[1]+j-1]==1:
          worldMap[loc[0]+i-1][loc[1]+j-1]=1
          flag=True
        elif mode=='search' and drone['state']=='goToTarget' and type(block)==str and block[0]=='S':
          continue
        elif type(block)==str and block[0]=='D' and block[1]!=id and (mode!='search' or drone['state']!='goToTarget'):
          drones[int(block[1])]['currentlyAbove']='S'+id
        elif type(block)!=str or block[0] not in ['V','D','B']:
          worldMap[loc[0]+i-1][loc[1]+j-1]='S'+id
  return flag

In [ ]:
def updateWorld():
  global worldMap
  global drones
  global currFrame
  spreadFire()
  for d in drones:
    if d['state']=='goToTarget':
      goToTarget(d)
    if d['state']=='findFire':
      findFire(d)
    if d['state']=='mapFire' or d['state']=='mapFire--init':
      mapFire(d)
  currFrame+=1


  #calc_drone_efficiencies()
  # estimate_ros()
  # if len(ros_estimates) > 0:
  #   print("ROS_estimates: " + str(ros_estimates[-1]))
    # if d['position']==d['target']:
    #   continue
    # if fireFound(d['position']):
    #   continue
    # worldMap[d['position'][0]][d['position'][1]]='V'+str(d['id'])
    # if abs(drones[i][0]-targets[i][0])>abs(drones[i][1]-targets[i][1]):
    #   if targets[i][0]>drones[i][0]:
    #     drones[i][0]=drones[i][0]+1
    #   else:
    #     drones[i][0]=drones[i][0]-1
    # else:
    #   if targets[i][1]>drones[i][1]:
    #     drones[i][1]=drones[i][1]+1
    #   else:
    #     drones[i][1]=drones[i][1]-1
    # worldMap[drones[i][0]][drones[i][1]]='D'+str(i)
  return worldMap

In [ ]:
def goToTarget(drone):
  global mode
  global mapTargets
  if mode=='search':
    clearPath(drone)
  if fireFound(drone):
    mode='map'
    drone['state']='mapFire--init'
    if mode!='map' or str(drone['currentlyAbove'])[0]!='B':
      mapTargets.append(drone['position'].copy())
      for d in drones:
        if d['id']!=drone['id'] and d['state']!='mapFire':
          d['state']='goToTarget'
          targetDistances=list(map(distance,[d['position']]*len(mapTargets),mapTargets))
          d['target']=mapTargets[targetDistances.index(min(targetDistances))].copy()
    if mode=='map' and drone['position']==drone['target']:
      removeIfExists(mapTargets,drone['target'])
    return
  if drone['position']==drone['target']:
    if mode=='search':
      clearPath(drone)
      drone['state']='findFire'
      drone['lastMove']='up'
    return
  if abs(drone['position'][0]-drone['target'][0])>abs(drone['position'][1]-drone['target'][1]):
    if drone['target'][0]>drone['position'][0]:
      move(drone,'right')
    else:
      move(drone,'left')
  else:
    if drone['target'][1]>drone['position'][1]:
      move(drone,'up')
    else:
      move(drone,'down')

In [ ]:
def clearPath(drone):
  id=str(drone['id'])
  for i in range(worldSize):
    for j in range(worldSize):
      if worldMap[i][j] in ['S'+id,'V'+id]:
        worldMap[i][j]=0

In [ ]:
def findFire(drone):
  global mode
  global mapTargets
  if fireFound(drone):
    mode='map'
    drone['state']='mapFire--init'
    mapTargets.append(drone['position'].copy())
    for d in drones:
      if d['id']!=drone['id'] and d['state']!='mapFire':
        d['state']='goToTarget'
        targetDistances=list(map(distance,[d['position']]*len(mapTargets),mapTargets))
        d['target']=mapTargets[targetDistances.index(min(targetDistances))].copy()
    return
  x=drone['position'][0]
  y=drone['position'][1]
  id=str(drone['id'])
  searchBounds=drone['searchBounds']
  if drone['lastMove']=='up' and (y+2>searchBounds[1][1] or (worldMap[x][y+2] in ['S'+id,'V'+id] and worldMap[x-1][y+2] in ['S'+id,'V'+id] and worldMap[x+1][y+2] in ['S'+id,'V'+id])):
    move(drone,'right')
  elif drone['lastMove']=='right' and (x+2>searchBounds[1][0] or (worldMap[x+2][y] in ['S'+id,'V'+id] and worldMap[x+2][y-1] in ['S'+id,'V'+id] and worldMap[x+2][y+1] in ['S'+id,'V'+id])):
    move(drone,'down')
  elif drone['lastMove']=='down' and (y-2<searchBounds[0][1] or (worldMap[x][y-2] in ['S'+id,'V'+id] and worldMap[x-1][y-2] in ['S'+id,'V'+id] and worldMap[x+1][y-2] in ['S'+id,'V'+id])):
    move(drone,'left')
  elif drone['lastMove']=='left' and (x-2<searchBounds[0][0] or (worldMap[x-2][y] in ['S'+id,'V'+id] and worldMap[x-2][y-1] in ['S'+id,'V'+id] and worldMap[x-2][y+1] in ['S'+id,'V'+id])):
    move(drone,'up')
  else:
    move(drone,drone['lastMove'])

In [ ]:
def mapFire(drone):
  global mode
  if mode=='map' and drone['position']==drone['target']:
    removeIfExists(mapTargets,drone['target'])
  x=drone['position'][0]
  y=drone['position'][1]
  visibility=[]
  for i in range(-1,2):
    visibility.append([])
    for j in range(-1,2):
      if x+i in range(0,worldSize) and y+j in range(0,worldSize) and world[x+i][y+j]==1:
        visibility[-1].append(1)
      else:
        visibility[-1].append(0)
  if visibility[1][1]==1:
    drone['currentlyAbove']=1
    visibility[1][1]=0
  prevVisibility=[]
  for i in range(-1,2):
    prevVisibility.append([])
    for j in range(-1,2):
      if x+i in range(0,worldSize) and y+j in range(0,worldSize) and worldMap[x+i][y+j]==1:
        prevVisibility[-1].append(1)
      else:
        prevVisibility[-1].append(0)
  if drone['lastMove']=='up':
    for i in range(3):
      for j in range(2):
        visibility[i][j]=visibility[i][j]&prevVisibility[i][j]
  if drone['lastMove']=='down':
    for i in range(3):
      for j in range(1,3):
        visibility[i][j]=visibility[i][j]&prevVisibility[i][j]
  if drone['lastMove']=='right':
    for i in range(2):
      for j in range(3):
        visibility[i][j]=visibility[i][j]&prevVisibility[i][j]
  if drone['lastMove']=='left':
    for i in range(1,3):
      for j in range(3):
        visibility[i][j]=visibility[i][j]&prevVisibility[i][j]
  fireFound(drone)
  # visibility[1][1]=0
  # if drone['lastMove']=='up':
  #   visibility[1][0]=0
  # if drone['lastMove']=='down':
  #   visibility[1][2]=0
  # if drone['lastMove']=='right':
  #   visibility[0][1]=0
  # if drone['lastMove']=='left':
  #   visibility[2][1]=0
  rotateDirections=['up','left','down','right']
  coordsOfDirection={'up':[x,y+1], 'down':[x,y-1], 'right':[x+1,y], 'left':[x-1,y]}
  # possibleDirections=['up','left','down','right']
  
  # for direction in rotateDirections:
  #   mat=np.rot90(np.array(visibility),rotateDirections.index(direction),(1,0)).tolist()
  #   left=rotateDirections[(rotateDirections.index(direction)+1)%4]
  #   opposite=rotateDirections[(rotateDirections.index(direction)+2)%4]
  #   right=rotateDirections[(rotateDirections.index(direction)+3)%4]
  #   print(direction)
  #   print(mat)
  #   mat=tuple(map(tuple, mat))
  #   if mat in mapDict:
  #     for d in set(rotateDirections)-set(mapDict[mat]):
  #       removeIfExists(possibleDirections,d)
  # possibleDirections=list(set(possibleDirections))
  # prevBlock=rotateDirections[(rotateDirections.index(drone['lastMove'])+2)%4]

  possibleDirections=[]
  if visibility[2][1]!=1 and 1 in np.array(visibility)[1:,:].reshape((6,1)) and (visibility[2][2]!=1 or visibility[2][0]!=1):
    possibleDirections.append('right')
  if visibility[0][1]!=1 and 1 in np.array(visibility)[:2,:].reshape((6,1)) and (visibility[0][2]!=1 or visibility[0][0]!=1):
    possibleDirections.append('left')
  if visibility[1][2]!=1 and 1 in np.array(visibility)[:,1:].reshape((6,1)) and (visibility[2][2]!=1 or visibility[0][2]!=1):
    possibleDirections.append('up')
  if visibility[1][0]!=1 and 1 in np.array(visibility)[:,:2].reshape((6,1)) and (visibility[2][0]!=1 or visibility[0][0]!=1):
    possibleDirections.append('down')

  if x==0:
    removeIfExists(possibleDirections,'left')
  if x==worldSize-1:
    removeIfExists(possibleDirections,'right')
  if y==0:
    removeIfExists(possibleDirections,'down')
  if y==worldSize-1:
    removeIfExists(possibleDirections,'up')
  
  if len(possibleDirections)>1 and 'lastMove' in drone:
    prevBlock=rotateDirections[(rotateDirections.index(drone['lastMove'])+2)%4]
    removeIfExists(possibleDirections,prevBlock)

  # print(np.array(visibility))
  # print(possibleDirections)

  nextDirection=''

  if len(possibleDirections)==1:
    #move(drone,possibleDirections[0])
    nextDirection=possibleDirections[0]
  elif len(possibleDirections)>1 and drone['state']=='mapFire--init':
    drone['state']='mapFire'
    direction1=possibleDirections[0]
    direction2=possibleDirections[1]
    block1=worldMap[coordsOfDirection[direction1][0]][coordsOfDirection[direction1][1]]
    block2=worldMap[coordsOfDirection[direction2][0]][coordsOfDirection[direction2][1]]
    prevDirection=rotateDirections.index(drone['lastMove'])
    if type(block1)==str and block1[0]=='B' and not (type(block2)==str and block2[0]=='B'):
      #move(drone,direction2)
      nextDirection=direction2
      #return
    elif type(block2)==str and block2[0]=='B' and not (type(block1)==str and block1[0]=='B'):
      #move(drone,direction1)
      nextDirection=direction1
      #return
    elif type(block1)==str and block1[0]=='B' and type(block2)==str and block2[0]=='B' and block1[1]==block2[1]:
      prevDrone=int(block1[1])
      if drones[prevDrone]['border'].index(coordsOfDirection[direction1])<drones[prevDrone]['border'].index(coordsOfDirection[direction2]):
        #move(drone,direction1)
        nextDirection=direction1
      else:
        #move(drone,direction2)
        nextDirection=direction2
      #return
    elif rotateDirections[(prevDirection+1)%4] in possibleDirections:
      #move(drone,rotateDirections[(prevDirection+1)%4])
      nextDirection=rotateDirections[(prevDirection+1)%4]
      #return
    else:
      #move(drone,drone['lastMove'])
      nextDirection=drone['lastMove']
    # for i in range(4):
    #   if rotateDirections[(prevDirection+i)%4] in possibleDirections:
    #     print(drone['lastMove'], rotateDirections[(prevDirection+i)%4], possibleDirections)
    #     move(drone,rotateDirections[(prevDirection+i)%4])
    #     return
  else:
    numBorders=[0,0,0,0]
    # print(possibleDirections)
    # print(drone['border'][-20:])
    for direction in possibleDirections:
      for box in drone['border'][-20:]:
        # if direction=='up' and box[1]<y:
        #   numBorders[0]+=1
        #   numBorders[2]-=1
        # if direction=='left' and box[0]>x:
        #   numBorders[1]+=1
        #   numBorders[3]-=1
        # if direction=='down' and box[1]>y:
        #   numBorders[2]+=1
        #   numBorders[0]-=1
        # if direction=='right' and box[0]<x:
        #   numBorders[3]+=1
        #   numBorders[1]-=1
        if direction=='up' or direction=='down':
          if box[1]<y:
            numBorders[0]+=1
            numBorders[2]-=1
          if box[1]>y:
            numBorders[2]+=1
            numBorders[0]-=1 
        if direction=='right' or direction=='left':
          if box[0]>x:
            numBorders[1]+=1
            numBorders[3]-=1
          if box[0]<x:
            numBorders[3]+=1
            numBorders[1]-=1
    for i in range(4):
      if rotateDirections[i] not in possibleDirections:
        numBorders[i]=float('-inf')
      if rotateDirections[i]=='up' and drone['position'][1]==worldSize-1:
        numBorders[i]=float('-inf')
      if rotateDirections[i]=='down' and drone['position'][1]==0:
        numBorders[i]=float('-inf')
      if rotateDirections[i]=='right' and drone['position'][0]==worldSize-1:
        numBorders[i]=float('-inf')
      if rotateDirections[i]=='left' and drone['position'][0]==0:
        numBorders[i]=float('-inf')
    #print(numBorders)
    #move(drone,rotateDirections[numBorders.index(max(numBorders))])
    # if max(numBorders)==float('-inf'):
    #   return
    nextDirection=rotateDirections[numBorders.index(max(numBorders))]
  if len(nextDirection)==0:
    print('SOMETHING IS WRONG')
    return
  nextBlock=worldMap[coordsOfDirection[nextDirection][0]][coordsOfDirection[nextDirection][1]]

  if type(nextBlock)==str and nextBlock[0]=='B':
    if coordsOfDirection[nextDirection] in drones[int(nextBlock[1])]['border'][-20:]:
      drone['followCount']+=1
    else:
      drone['followCount']=0
  else:
    drone['followCount']=0
  
  #print(drone['id'],nextBlock,drone['followCount'],drone['position'])

  if drone['followCount']==12 or (type(drone['currentlyAbove'])==str and drone['currentlyAbove'][0]=='D'):
    drone['followCount']=0
    drone['lastMove']=rotateDirections[(rotateDirections.index(nextDirection)+2)%4]
    return

  # if type(nextBlock)==str and type(drone['currentlyAbove'])==str and nextBlock==drone['currentlyAbove'] and nextBlock[0]=='B' and nextBlock[1]!=drone['id']:
  #   dId=int(nextBlock[1])
  #   if coordsOfDirection[nextDirection] in drones[dId]['border'][-20:] and drone['position'] in drones[dId]['border'][-20:]:
  #     if drones[dId]['border'].index(drone['position']) < drones[dId]['border'].index(coordsOfDirection[nextDirection]):
  #       drone['lastMove']=rotateDirections[(rotateDirections.index(nextDirection)+2)%4]
  #       print('IT HAPPENED',drone['id'],drone['position'])
  #       return
  move(drone,nextDirection)

In [ ]:
def removeIfExists(arr,elem):
  if elem in arr:
    arr.remove(elem)

In [ ]:
def move(drone, direction):
  global mode
  if mode=='search' and drone['state']=='goToTarget':
    worldMap[drone['position'][0]][drone['position'][1]]=drone['currentlyAbove']#'V'+str(drone['id'])
  elif mode=='map' and drone['state']=='mapFire':
    drone['border'].append(drone['position'].copy())
    worldMap[drone['position'][0]][drone['position'][1]]='B'+str(drone['id'])
  else:
    worldMap[drone['position'][0]][drone['position'][1]]='V'+str(drone['id'])
  if drone['currentlyAbove']==1:
    world[drone['position'][0]][drone['position'][1]]=1
  else:
    world[drone['position'][0]][drone['position'][1]]='V'+str(drone['id'])
  if direction=='left':
    drone['position'][0]=drone['position'][0]-1
    drone['lastMove']='left'
  if direction=='right':
    drone['position'][0]=drone['position'][0]+1
    drone['lastMove']='right'
  if direction=='up':
    drone['position'][1]=drone['position'][1]+1
    drone['lastMove']='up'
  if direction=='down':
    drone['position'][1]=drone['position'][1]-1
    drone['lastMove']='down'
  nextSquare=worldMap[drone['position'][0]][drone['position'][1]]
  while type(nextSquare)==str and nextSquare[0]=='D':
    nextSquare=drones[int(nextSquare[1])]['currentlyAbove']
  drone['currentlyAbove']=nextSquare
  worldMap[drone['position'][0]][drone['position'][1]]='D'+str(drone['id'])
  world[drone['position'][0]][drone['position'][1]]='D'+str(drone['id'])

In [ ]:
def spreadFire():
  global world
  global adjList
  global spread_mode
  global perimeter_coverage
  global soft_perimeter_coverage
  
  if spreadMode == 'random':
    for i in range(len(adjList)//50):
      newFire=adjList[random.randrange(0,len(adjList))]
      world[newFire[0]][newFire[1]]=1
      adjList.remove(newFire)
      allFires.append([newFire[0],newFire[1]])
      if newFire[0]>1 and world[newFire[0]-1][newFire[1]]!=1 and ([newFire[0]-1,newFire[1],prob_spread_head] not in adjList and [newFire[0]-1,newFire[1],prob_spread_flanks] not in adjList):
        adjList.append([newFire[0]-1,newFire[1],prob_spread_flanks])
      if newFire[0]<worldSize-2 and world[newFire[0]+1][newFire[1]]!=1 and ([newFire[0]+1,newFire[1],prob_spread_head] not in adjList and [newFire[0]+1,newFire[1],prob_spread_flanks] not in adjList):
        adjList.append([newFire[0]+1,newFire[1],prob_spread_flanks])
      if newFire[1]>1 and world[newFire[0]][newFire[1]-1]!=1 and ([newFire[0],newFire[1]-1, prob_spread_head] not in adjList and [newFire[0],newFire[1]-1,prob_spread_flanks] not in adjList):
        adjList.append([newFire[0],newFire[1]-1,0])
      if newFire[1]<worldSize-2 and world[newFire[0]][newFire[1]+1]!=1 and ([newFire[0],newFire[1]+1,prob_spread_head] not in adjList and [newFire[0],newFire[1]+1,prob_spread_flanks]):
        adjList.append([newFire[0],newFire[1]+1,prob_spread_head])
      updatePerimeter([newFire[0], newFire[1]])
      updatePerimeter([newFire[0]-1, newFire[1]])
      updatePerimeter([newFire[0]+1, newFire[1]])
      updatePerimeter([newFire[0], newFire[1]-1])
      updatePerimeter([newFire[0], newFire[1]+1])

  elif spreadMode == 'ellipse':
    tempAdjList = []
    for newFire in adjList:
      #newFire=adjList[random.randrange(0,len(adjList))]
      randNum = random.random()
      if randNum < newFire[2]:
        world[newFire[0]][newFire[1]]=1
        adjList.remove(newFire)
        allFires.append([newFire[0],newFire[1]])
        if newFire[0]>1 and world[newFire[0]-1][newFire[1]]!=1 and ([newFire[0]-1,newFire[1],prob_spread_head] not in adjList and [newFire[0]-1,newFire[1],prob_spread_flanks] not in adjList):
          tempAdjList.append([newFire[0]-1,newFire[1],prob_spread_flanks])
        if newFire[0]<worldSize-2 and world[newFire[0]+1][newFire[1]]!=1 and ([newFire[0]+1,newFire[1],prob_spread_head] not in adjList and [newFire[0]+1,newFire[1],prob_spread_flanks] not in adjList):
          tempAdjList.append([newFire[0]+1,newFire[1],prob_spread_flanks])
        if newFire[1]>1 and world[newFire[0]][newFire[1]-1]!=1 and ([newFire[0],newFire[1]-1, prob_spread_head] not in adjList and [newFire[0],newFire[1]-1,prob_spread_flanks] not in adjList):
          tempAdjList.append([newFire[0],newFire[1]-1,0])
        if newFire[1]<worldSize-2 and world[newFire[0]][newFire[1]+1]!=1 and ([newFire[0],newFire[1]+1,prob_spread_head] not in adjList and [newFire[0],newFire[1]+1,prob_spread_flanks]):
          tempAdjList.append([newFire[0],newFire[1]+1,prob_spread_head])
        updatePerimeter([newFire[0], newFire[1]])
        updatePerimeter([newFire[0]-1, newFire[1]])
        updatePerimeter([newFire[0]+1, newFire[1]])
        updatePerimeter([newFire[0], newFire[1]-1])
        updatePerimeter([newFire[0], newFire[1]+1])
  
  adjList.extend(tempAdjList)
  

  npWorld=np.array(world)
  for i in range(worldSize):
    for j in range(worldSize):
      if world[i][j]==0 and 1 in npWorld[0:i,j] and 1 in npWorld[i:worldSize,j] and 1 in npWorld[i,0:j] and 1 in npWorld[i,j:worldSize]:
        world[i][j]=1
  npWorld=np.array(world)
  #plot(world)

In [ ]:
def estimate_ros(): #don't call this function before createWorld()
  global ros_ref_points
  global last_ros_update
  global ros_estimates
  global firesDiscovered

  if len(firesDiscovered) > 0:
    if time_since_drone_spawn() > 0.5 and len(ros_ref_points) == 0: #should be after the initial perimeter's been established
      ros_ref_points.append(max(firesDiscovered, key=lambda x: x[0]))
      ros_ref_points.append(min(firesDiscovered, key=lambda x: x[0]))
      ros_ref_points.append(max(firesDiscovered, key=lambda x: x[1]))
      ros_ref_points.append(min(firesDiscovered, key=lambda x: x[1]))
      last_ros_update = time_since_drone_spawn()

    time_since_last_update = time_since_drone_spawn() - last_ros_update 
    if time_since_last_update > 0.25 and last_ros_update != 0: 
      new_rightmost = max(firesDiscovered, key=lambda x: x[0])
      new_leftmost = min(firesDiscovered, key=lambda x: x[0])
      new_uppermost = max(firesDiscovered, key=lambda x: x[1])
      new_lowermost = min(firesDiscovered, key=lambda x: x[1])


      ros_rightflank = (new_rightmost[0] - ros_ref_points[0][0])*km_per_square/time_since_last_update
      ros_leftflank = abs((new_leftmost[0] - ros_ref_points[1][0]))*km_per_square/time_since_last_update
      ros_head = (new_uppermost[1] - ros_ref_points[2][1])*km_per_square/time_since_last_update
      ros_back = abs((new_lowermost[1] - ros_ref_points[3][1]))*km_per_square/time_since_last_update

      ros_estimates.append([ros_rightflank,ros_leftflank,ros_head,ros_back])
      
      ros_ref_points.clear()
      ros_ref_points.append(new_rightmost)
      ros_ref_points.append(new_leftmost)
      ros_ref_points.append(new_uppermost)
      ros_ref_points.append(new_lowermost)

      last_ros_update = time_since_drone_spawn()



In [ ]:
def actual_ros():
  global actual_ros_ref_points
  global last_actual_ros_update
  global actual_ros_estimates

  if len(actual_ros_ref_points) == 0: #should be after the initial perimeter's been established
    actual_ros_ref_points.append(max(allFires, key=lambda x: x[0]))
    actual_ros_ref_points.append(min(allFires, key=lambda x: x[0]))
    actual_ros_ref_points.append(max(allFires, key=lambda x: x[1]))
    actual_ros_ref_points.append(min(allFires, key=lambda x: x[1]))
    last_actual_ros_update = time_since_drone_spawn()

  time_since_last_update = time_since_drone_spawn() - last_actual_ros_update
  if time_since_last_update > 0.5: 
    new_rightmost = max(allFires, key=lambda x: x[0])
    new_leftmost = min(allFires, key=lambda x: x[0])
    new_uppermost = max(allFires, key=lambda x: x[1])
    new_lowermost = min(allFires, key=lambda x: x[1])

    ros_rightflank = (new_rightmost[0] - actual_ros_ref_points[0][0])*km_per_square/time_since_last_update
    ros_leftflank = abs((new_leftmost[0] - actual_ros_ref_points[1][0]))*km_per_square/time_since_last_update
    ros_head = (new_uppermost[1] - actual_ros_ref_points[2][1])*km_per_square/time_since_last_update
    ros_back = abs((new_lowermost[1] - actual_ros_ref_points[3][1]))*km_per_square/time_since_last_update

    actual_ros_estimates.append([ros_rightflank,ros_leftflank,ros_head,ros_back])
    
    actual_ros_ref_points.clear()
    actual_ros_ref_points.append(new_rightmost)
    actual_ros_ref_points.append(new_leftmost)
    actual_ros_ref_points.append(new_uppermost)
    actual_ros_ref_points.append(new_lowermost)

    last_actual_ros_update = time_since_drone_spawn()



In [ ]:
def calc_drone_efficiencies(): 
  global efficiencies
  curr_efficiencies = {}

  for drone in drones:
    if len(firesDiscovered) == 0:
      curr_efficiencies[drone['id']] = 0
    else:
      curr_efficiencies[drone['id']] = round(100*drone['numDiscovered']/len(firesDiscovered))
  efficiencies.append(sorted(curr_efficiencies.items(), key=lambda x: x[1], reverse=True))
  

In [ ]:
def time_since_drone_spawn():
  return currFrame*frame_time_millis*speedup_factor/millis_per_hr #elapsed time in hours

In [ ]:
def updatePerimeter(x): 
  global perimeter
  if world[x[0]][x[1]] == 1:
    if world[x[0]-1][x[1]] == 0 or world[x[0]+1][x[1]] == 0 or world[x[0]][x[1]-1] == 0 or world[x[0]][x[1]+1] == 0:
      if [x[0],x[1]] not in perimeter:
        perimeter.append([x[0],x[1]])
    else:
      if [x[0],x[1]] in perimeter:
        perimeter.remove([x[0],x[1]])

In [ ]:
def percent_perimeter_detected():
  global perimeter
  global perimeter_coverage
  numDetected = 0
  for fire in perimeter:
    if worldMap[fire[0]][fire[1]] == 1: #fire on perimeter was detected by drone
      numDetected += 1
  perimeter_coverage.append(round(100*numDetected/len(perimeter)))
  return 100*numDetected/len(perimeter)

In [ ]:
def percent_soft_perimeter_detected():
  global perimeter
  global soft_perimeter_coverage
  numDetected = 0
  for fire in perimeter:
    if worldMap[fire[0]][fire[1]] == 1 or worldMap[fire[0]-1][fire[1]] == 1 or worldMap[fire[0]+1][fire[1]] == 1 or worldMap[fire[0]][fire[1]-1] == 1 or worldMap[fire[0]][fire[1]+1] == 1 or worldMap[fire[0]-1][fire[1]-1] == 1 or worldMap[fire[0]+1][fire[1]+1] == 1 or worldMap[fire[0]-1][fire[1]+1] == 1 or worldMap[fire[0]+1][fire[1]-1] == 1:
      numDetected += 1
  soft_perimeter_coverage.append(round(100*numDetected/len(perimeter)))
  return 100*numDetected/len(perimeter)


In [ ]:
def drawframe(n):
  global worldMap
  global image1
  global image2
  global textBoxes
  global actual_ros_estimates
  global ros_estimates
  updateWorld()
  #plot(world)
  image1.set_data(convert(world))
  image2.set_data(convert(worldMap))

  percent_perimeter_detected()
  #print("Percent perimeter detected: " + str(perimeter_coverage[-1]))
  percent_soft_perimeter_detected()
  #print("Percent perimeter detected (soft): " + str(soft_perimeter_coverage[-1]))
  #print("Total fires discovered: " + str(len(firesDiscovered)))
  #plt.figtext(0,0.23,"Total fires discovered: " + str(len(firesDiscovered)))
  calc_drone_efficiencies()
  #print("Drone efficiencies: " + str(efficiencies[-1]))
  estimate_ros()
  actual_ros()

  textBoxes[0].set_text("Minutes since drones spawned: " + str(round(60*time_since_drone_spawn())))
  textBoxes[1].set_text("Wind speed (km/h): " + str(windSpeed))
  textBoxes[2].set_text("Percent perimeter detected: " + str(perimeter_coverage[-1]))
  textBoxes[3].set_text("Percent perimeter detected (soft): " + str(soft_perimeter_coverage[-1]))
  textBoxes[4].set_text("Drone efficiencies: " + str(efficiencies[-1]))
  textBoxes[5].set_text("ROS estimates: " + str(ros_estimates[-1]) if len(ros_estimates) > 0 else 'ROS estimates: Collecting Data . . .')
  textBoxes[6].set_text("Actual ROS: " + str(actual_ros_estimates[-1]) if len(actual_ros_estimates) > 0 else 'Actual ROS: Collecting Data . . .')

def convert(grid):
  img=copy.deepcopy(grid)
  img=np.array(img).T.tolist()
  for i in range(len(img)):
    for j in range(len(img)):
      if img[i][j]==1 or img[i][j]=='1': # 1 -> Fire
        img[i][j]=[255,0,0]
      elif type(img[i][j])==str and img[i][j][0]=='D': # D -> Drone
        img[i][j]=[0,0,255]
      elif type(img[i][j])==str and img[i][j][0]=='T': # T -> Target
        img[i][j]=[0,255,255]
      elif type(img[i][j])==str and img[i][j][0]=='V': # V -> Visited
        img[i][j]=[150,150,150]
      elif type(img[i][j])==str and img[i][j][0]=='S': # S -> Seen
        img[i][j]=[210,210,210]
      elif type(img[i][j])==str and img[i][j][0]=='B': # B -> Border
        img[i][j]=[0,255,0]
      else:
        img[i][j]=[255,255,255]
  return img

def runSimulation():
  global worldMap
  global anim
  global image1
  global image2
  global textBoxes
  size=2
  fig=plt.figure(figsize=[6.4*size,4.8*size],dpi=100)
  ax=plt.subplot(1,2,1)
  ax.title.set_text("Ground Truth")
  bx=plt.subplot(1,2,2)
  bx.title.set_text("Drone's Perspective")
  image1=ax.imshow(convert(world),origin="lower")
  image2=bx.imshow(convert(worldMap),origin="lower")
  textBoxes=[
             plt.figtext(0.4,0.78,'Minutes since drones spawned'),
             plt.figtext(0.1,0.23,'Wind speed'),
             plt.figtext(0.1,0.21,'Percent Perimeter Detected'),
             plt.figtext(0.1,0.19,'Percent Perimeter Detected (soft)'),
             plt.figtext(0.1,0.17,'Drone Efficiencies'),
             plt.figtext(0.1,0.15,'ROS estimates'),
             plt.figtext(0.1,0.13,'ROS actual'),
             
  ]
  anim = animation.FuncAnimation(fig, drawframe, frames=400, interval=frame_time_millis, blit=False)
  #HTML(anim.to_html5_video())

In [ ]:
batch_run()

In [ ]:
# anim.save('smallFire4.mp4')
# from google.colab import files
# files.download('smallFire4.mp4')

In [ ]:
# print("Actual World")
# createWorld()
# spawnDrones()
# plot(world)
# print("World from Drone's view")
# createWorldMap()
# worldSplit()
# print("Targets")
# setInitTargets()
# print("Init drones")
# for i in drones:
#   print(i)
# print("Simulation")
# runSimulation()
# HTML(anim.to_html5_video())